<a href="https://colab.research.google.com/github/PMartey/KaggleX-Project/blob/main/Evaluating_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd

In [ ]:
# Load the fine tuned model that was just uploaded to Kaggle

kaggle_username = "dedemartey"
finetuned_model = keras_nlp.models.CausalLM.from_preset(f"kaggle://{kaggle_username}/gemma/keras/finbot_my_gemma2_pt")



100%|██████████| 785/785 [00:00<00:00, 1.07MB/s]


100%|██████████| 2.91k/2.91k [00:00<00:00, 1.23MB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 8.77MB/s]


100%|██████████| 9.34G/9.34G [03:30<00:00, 47.7MB/s]


#Inference after fine-tuning  
After fine-tuning, responses follow the instruction provided in the prompt.


#Loss Harvesting Prompt

In [ ]:
test_df= pd.read_csv('/content/test_data.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2582 entries, 0 to 2581
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  2582 non-null   object
 1   answer    2582 non-null   object
dtypes: object(2)
memory usage: 40.5+ KB


In [ ]:
test_samp =test_df.sample(25)

3. Evaluate using Quantitative Metrics  
BLEU Score: Measures the similarity between generated answers and reference answers by comparing n-grams. BLEU is widely used for translation and question-answering tasks, but note that it’s biased toward exact matches.  
ROUGE Score: Compares overlapping n-grams, especially for longer text. ROUGE is particularly useful for assessing responses with more paraphrasing.  
Exact Match (EM): Measures the percentage of predictions that exactly match the ground truth.  
F1 Score: Useful when exact wording isn’t required. It’s a harmonic mean of precision and recall, especially effective for partial matches.   


In [ ]:
# Example: Generate predictions for each question in the test set
predictions = []
for question in test_samp['question']:
    generated_answer = finetuned_model.generate(question)
    predictions.append(generated_answer)


In [ ]:
!pip install nltk

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=cbc3ac09feb6e338549eba240d429d70a256c1720604ddb1e829f6a7de46f269
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Calculate BLEU and ROUGE for each generated answer
bleu_scores = []
# Remove the smoothing_function argument from RougeScorer initialization
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge_scores = []

for ref, pred in zip(test_samp['answer'], predictions):
    # Use smoothing function for BLEU calculation
    bleu_scores.append(sentence_bleu([ref.split()], pred.split(), smoothing_function=SmoothingFunction().method1))
    rouge_scores.append(rouge.score(ref, pred))

# Reason for using the smoothing function

If your model produces outputs that differ significantly from the reference answers, even if they capture the general idea, the BLEU score might still be low due to this lack of n-gram overlap. Here’s how you can handle it:

Lower the n-gram order: Reduce the BLEU score calculation to 1-gram or 2-gram BLEU, which focuses more on individual word overlaps rather than multi-word phrases.

Use a smoothing function: Applying smoothing (e.g., SmoothingFunction().method1 in NLTK) can help avoid zero scores, especially for shorter sentences or those with uncommon n-grams.

In [ ]:
# Calculate average ROUGE scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print(f"Average ROUGE-1: {avg_rouge1}")

Average ROUGE-1: 0.2173700398733314


In [ ]:
# Calculate average BLEU Scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {avg_bleu}")

Average BLEU Score: 0.005526892104409539
